# <center>Parallel Processing with Apache Spark</center>

Apache Spark is probably the most popular open-source big data platform for data science. Unlike Hadoop MapReduce which reads and writes data to disk, Spark emphasises working in memory, enhancing computation speed. With Spark, you can distribute data and perform computations across clusters comprising multiple nodes, where each node can be envisioned as an individual computer. Partitioning your data into smaller portions using this approach simplifies handling of extensive datasets, as each node only processes a limited quantity of data. Spark can be used for a variety of use cases, but there are some situations where it may not be the best choice.

<u>Spark Limitations</u>
1. Small-scale data processing: Spark is designed to process large volumes of data and is not optimal for working with small datasets or performing simple data processing tasks such as filtering, sorting, or aggregating data. 
2. Collaboration: Spark runs on clusters of machines, which can make it challenging for teams to collaborate effectively. Teams may face issues related to version control, conflicting code changes, and communication gaps.
3. Cloud: Spark itself is not specifically optimized for the cloud, and there are certain challenges that can arise when deploying Spark in a cloud-based environment.
4. Security: If you have strict data security requirements, Spark may not be the best choice due to its distributed nature.

## Spark DataFrame vs Pandas DataFrame

Spark DataFrame and Pandas DataFrame are both tabular data structures that organize data into rows and columns, but they have some key differences that make them more suitable for different types of data processing scenarios. One of the main differences is that **Spark DataFrame is designed for distributed computing** across a cluster of machines, while pandas DataFrame is designed for single-machine processing. Spark DataFrame is optimized for processing large-scale data that may not fit into the memory of a single machine, while pandas DataFrame is typically used for smaller-scale data that can fit into memory.

Another difference is that **Spark DataFrame is immutable**, meaning that once it is created, its contents cannot be modified. This makes it easier to reason about data transformations in a distributed environment. Transformations on an immutable DataFrame can be performed by creating a new DataFrame with the transformed data. Since the original DataFrame is immutable and cannot be modified, each transformation creates a new DataFrame instance. In contrast, pandas DataFrame is mutable, meaning that its contents can be modified in place.

In terms of functionality, Spark DataFrame supports a wide range of operations, including SQL-style queries, filtering, aggregation, and machine learning algorithms. However, for certain types of operations, Spark DataFrame is  slower than pandas DataFrame due to the overhead of distributed computing. In such cases, if Pandas cannot handle the workload, one may consider using Dask.

<u>Good to know</u><br>

Spark DataFrames are built on top of RDDs (Resilient Distributed Datasets), which are the fundamental data structure in Apache Spark. RDDs are a distributed collection of objects that can be processed in parallel across a cluster of machines. Spark DataFrame is a more high-level abstraction that provides a more familiar and concise API for data processing, similar to the dataframes in pandas. RDDs can still be used directly in Spark, but they are more low-level and flexible, and require more manual coding compared to using DataFrames. RDDs are suitable for processing unstructured or semi-structured data and custom algorithms that cannot be expressed easily using the DataFrame API. However, for most data processing tasks, Spark DataFrames are the preferred choice due to their more efficient and concise API.

## Getting Started with PySpark DataFrames

To use Spark, the first step after installing the PySpark library is to connect to a cluster. The cluster is usually hosted on a remote machine and comprises a master computer that manages data and computation splitting, and worker computers that receive and process data and calculations. For beginners, it is easier to run a local cluster.

To connect to the cluster, we need to create an instance of the `SparkSession` class, which acts as a unified entry point for working with Spark. This `SparkSession` object includes a `SparkContext` instance, which is used to manage the connection to the cluster. Think of the `SparkContext` as the underlying connection to the cluster, and the `SparkSession` as the main interface to that connection. 

In [2]:
# Import necessary PySpark modules
from pyspark.sql import SparkSession

# Create a SparkSession object
spark = SparkSession.builder.appName("example-app").getOrCreate()

- `SparkSession`: A class in PySpark used to create a SparkSession object.
- `builder`: A method of the SparkSession class used to create a Builder object.
- `appName()`: A method of the Builder object used to set the name of the application to be created. This name will be shown in the Spark Web UI and logs for easy identification.
- `"example-app`": The name given to the application.
- `getOrCreate()`: A method of the Builder object used to create a SparkSession object with the specified configuration, or return an existing SparkSession object if one already exists.

Once we have created a `SparkSession`, can start working with Spark DataFrames. We can create DataFrames directly from various sources such as CSV, JSON, or Parquet files, or by converting from existing RDDs using the `toDF()` method. We can also convert a Pandas DataFrame to a Spark DataFrame using the spark `.createDataFrame()` method and passing to it the Pandas DataFrame object. 

Let's start small by reading in a CSV file using `.read.csv()`. Other than providing the path to the data, the parameters that we are interested in are `header` and `inferSchema`. The `header` parameter specifies whether the first row of the CSV file should be used as the column names for the resulting dataframe. The `inferSchema` parameter specifies whether Pyspark should automatically infer the schema (data types) of the columns based on the data in the CSV file.

In [6]:
# Import csv as spark dataframe
df = spark.read.csv("C:\\Users\\ihadjimpalasis\\Desktop\\Projects\\cyta\\Datasets\\business_final.csv", header=True, inferSchema=True)

#### The catalog

A Spark DataFrame allows for SQL querying when the table is registered in the catalog. The PySpark catalog is a place where information about data and metadata objects is stored. Data objects refer to actual data sets that are stored in a distributed file system, such as Apache Hadoop HDFS, or in a cloud-based storage system like Amazon S3 or Google Cloud Storage. On the other hand, metadata objects refer to information about data objects, such as the schema, partitioning, and data location. 

**Examples of data objects:**
1. DataFrames - A distributed collection of data organized into named columns, similar to a table in a relational database.
2. RDDs (Resilient Distributed Datasets) - A distributed collection of immutable data objects that can be processed in parallel across a cluster.
3. Dataset - A strongly typed distributed collection of data that allows for compile-time type checking and provides a more efficient API compared to RDDs.

**Examples of metadata objects:**
1. Table schema - Defines the structure of a table or DataFrame, including column names, data types, and other constraints.
2. Partitioning scheme - Specifies how data is partitioned across a cluster, which can affect query performance.
3. Data source information - Provides details about where data is stored, such as file format, location, and other configuration options.
4. View definition - Specifies the SQL query used to define a view, which is a virtual table that can be queried like a regular table.

**<u>Common uses of the catalog:</u>**
- *Registering new data sources*: You can use the `createOrReplaceTempView()` method to register a new data source in the catalog.
- *Listing databases and tables*: You can use the `listDatabases()` and `listTables()` methods to get a list of all the databases and tables available in the current `SparkSession`.
- *Get table metadata*: You can use the `Table` class to query metadata about a specific table, such as the schema, partitioning, and location.
- *Creating, dropping, or altering tables and views*: You can use the `createTable()`, `dropTable()`, and `alterTable()` methods to create, drop, or alter tables and views in the catalog.
- *Caching or uncaching data*: You can use the `cacheTable()` and `uncacheTable()` methods to cache or uncache a DataFrame or table in memory, which can improve query performance.

In [12]:
# Register dataframe as a temporary view
df.createOrReplaceTempView("df_temp_view")

In [ ]:
# Run a SQL query on the view
query = "SELECT * FROM df_temp_view LIMIT 5"
spark.sql(query)

Registering a dataframe as a temporary view creates a SQL view on top of the dataframe. A temporary view allows you to query and manipulate data using SQL syntax and is only available for the duration of the SparkSession that created it. Temporary views can be useful for quickly querying and exploring data without having to write complex DataFrame operations. They can also be used in conjunction with other Spark SQL functionality, such as window functions and joins, to perform more complex analysis.

The other type of view in PySpark is global temporary view - `.createGlobalTempView("my_global_view")` -  which is accessible across all nodes in the Spark cluster. This means that any `SparkSession` running on any node in the cluster can access a global temporary view as long as it has the appropriate permissions. When you create a global temporary view, it is stored in a system-defined database called `global_temp` in the Spark catalog. The view is registered with a name that is prefixed with "global_temp.". For example, if you create a global temporary view with the name "my_global_view", its fully qualified name in the Spark catalog will be "global_temp.my_global_view".

In [15]:
# Get list of tables in cluster
tables = spark.catalog.listTables()

# print table names
for table in tables:
    print(table.name)

df_temp_view


In [ ]:
# Get table metadata
schema = spark.table("df_temp_view").schema
columns = spark.table("df_temp_view").columns
partitioning = spark.table("df_temp_view").rdd.getNumPartitions()
location = spark.table("df_temp_view").storageLevel.useDisk

In [31]:
# Drop the temporary view
spark.catalog.dropTempView("df_temp_view")

True

In [ ]:
# Cache DataFrame to memory
df.cache()

Caching a DataFrame in memory means storing the DataFrame's data in memory on the executor nodes of a Spark cluster so that it can be quickly accessed in subsequent Spark actions or transformations. Caching can significantly improve the performance of repetitive operations on the same DataFrame since the data does not need to be read from disk or computed again from scratch. Instead, the data is already available in memory and can be retrieved quickly. Un-caching a DataFrame means removing the cached data from memory, freeing up memory resources on the executor nodes.

There is no point in uncaching a Spark DataFrame that has not been explicitly cached using the `.cache()` method. Spark automatically manages the caching of DataFrames and will automatically remove them from memory when necessary. In PySpark, we can cache a DataFrame using the `.cache()` method, and uncache using the `.unpersist()` method.